In [ ]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

from random import randint , choice
from bs4 import BeautifulSoup
from datetime import datetime

import pandas as pd
import undetected_chromedriver as uc
import sys,shutil, os,time ,multiprocessing






class customselenium():
    __ver__ = '2.0' #Version of Custom Selenium 
    __dev__ = 'Sameer K' # Developed By 


    def __init__(self) -> None:      
        self.wait = None
        self.x = 0
        self.y = 0
        self.driver = None
        self.file = self.__class__.__name__
        self.random_user_agent = self.random_user() 
        self.chrome_ext = None
        self.lock =  multiprocessing.Manager().Lock()
        
    def intialize_driver(self):
        if self.driver is None:
            self.driver = self.browser()
            self.wait = WebDriverWait(self.driver,30)

    def close(self):
        if self.driver:
            self.driver.quit()
            self.driver = None
            self.wait = None
    
    def readFile(self,file):
        file,ext = file.split('.')
        data_frame = pd.DataFrame()
        try:
            if ext == 'csv':
                data_frame = pd.read_csv(f"{file}.{ext}")
            elif ext == 'xlsx':
                data_frame = pd.read_excel(f"{file}.{ext}")
        except Exception as ex:
            pass

        return data_frame         
   
    def browser(self,head=False):                  
        options = uc.ChromeOptions()
        
        # options.add_argument(f"--headers={headers}")
        options.add_argument("--disable-popup-blocking")
        # options.add_argument("--force-device-scale-factor=0.8")

        # options.add_experimental_option('prefs' ,{
        #     "download.default_directory": os.path.join(os.getcwd() , "CaptchaImage")
        # })

        # if not os.path.exists(os.path.join(os.getcwd() , "CaptchaImage")):
        #     os.mkdir(os.path.join(os.getcwd() , "CaptchaImage"))

        if self.chrome_ext is not None:
            options.add_argument(f'--load-extension={self.chrome_ext}')  

        # browser_executable_path = ".\Google\Chrome\Application\chrome.exe"

        # if not os.path.exists(browser_executable_path):
        #     print('Chrome Folder Missing, make sure chrome folder exist in current working directory.')
        #     input()
        #     sys.exit(0)
            

        try:
            driver = uc.Chrome(
                options=options,
                use_subprocess=True,
                headless=head,
                # browser_executable_path=browser_executable_path
            )            
        except Exception as ex:            
            print('Chrome Intializing Error...probably Version error \n') 
            input()          
            sys.exit(0)   

        driver.maximize_window()
        print('Chrome Intialized...')
        return driver
            
    def get_Xpath_value(self,xpath_Id,all_elements=False,wait_time = 15):        
        self.wait = WebDriverWait(self.driver,wait_time)
        if all_elements:
            locate_element = EC.presence_of_all_elements_located
        else:
            locate_element = EC.presence_of_element_located        
        try:
            value = self.wait.until(locate_element((By.XPATH, xpath_Id)))
        except Exception as ex:
            value = None   
        return value    
    
    def datime(self):
        return datetime.today().timestamp()   
    
    def chunk_list(self, data, num_chunks):
        chunk_size, remainder = divmod(len(data), num_chunks)
        chunks = []
        start = 0

        for i in range(num_chunks):
            if i < remainder:
                end = start + chunk_size + 1
            else:
                end = start + chunk_size

            chunks.append(data[start:end])
            start = end

        return chunks
    
    def remove_file(self):
        try:
            os.remove(f"{self.file}") if "xlsx" in self.file else os.remove(f"{self.file}.xlsx")    
        except Exception as ex:
            pass
    
    def randomSleep(self,start,end):        
        time.sleep(randint(start,end))

    def openNewWindow(self):
        try:
            self.driver.execute_script("window.open(arguments[0], '_blank')")
            self.driver.switch_to.window(self.driver.window_handles[-1])
        except Exception as ex:
            print(ex)
            print("Driver is not available. Make sure the driver is started.")
        return self.driver
    
    def save(self,name = None,trig=False,**kwargs):
        if name:
            self.file = name  

        try:
            dataframe = pd.read_excel(f"{self.file}.xlsx")
        except FileNotFoundError:
            dataframe = pd.DataFrame()        
        
        if trig:
            dataframe1 = pd.DataFrame(kwargs)
        else:
            dataframe1 = pd.DataFrame([kwargs])

        dataframe = pd.concat([dataframe,dataframe1],ignore_index=True)
        dataframe.to_excel(f"{self.file}.xlsx",index=False)

    def click(self,xpath,wait_time=15):
        self.wait = WebDriverWait(self.driver,wait_time)
        try:
            element = self.wait.until(EC.presence_of_element_located((By.XPATH,xpath)))
            self.scrollToElement(element=element)
            self.randomSleep(2,3)
            element.click()
            return True
        except Exception as ex:
            return False
            
    def get_page_source(self,element=None):
        try:
            if element:
                source = element.get_attribute("innerHTML")
            else:
                source = self.driver.page_source              
            soup = BeautifulSoup(source,'lxml')            
        except Exception as ex : 
            soup = None
        
        return soup

    def scrollToElement(self,element):     

            
        try:
            self.driver.execute_script(
                    "arguments[0].scrollIntoView({behavior: 'auto', block: 'center', inline: 'center'});",
                    element,
            )
            

            
            x = element.location.get('x')
            y = element.location.get('y')
            
            if (self.x == x) and (self.y == y) :
                return False
        
            self.x = x
            self.y = y   
            
            return True
        except:
            return False
        
        
    
    def random_user(self):
        browsers = (
            "Mozilla",
            "Opera",
            "Internet Explorer",
            "Chrome",
            "Safari",
        )

        operating_system = (
            "Windows NT 10.0; Win64; x64",
            "Windows NT 6.1; WOW64",
            "Macintosh; Intel Mac OS X 10_15_4",
            "X11; Linux x86_64",
            "Android 10.0; Mobile",
            "iOS 13.4; iPhone",
        )

        version = {
            "Mozilla": f"{randint(3, 15)}.0",
            "Opera": f"{randint(3, 15)}.80",
            "Internet Explorer": f"{randint(3, 15)}.0",
            "Chrome": f"{randint(70, 110)}.0.3987.{randint(100, 999)}",
            "Safari": f"{randint(500, 550)}.36",
        }

        browser = choice(browsers)
        os = choice(operating_system)
        browser_version = version[browser]

        user_agent = f"{browser}/{browser_version} ({os})"

        return user_agent
    
    def validateData(self,element):
        if element :
            return element.text
        return None
        
    def load_extension(self,name,version):
        self.chrome_ext = os.path.join(os.getcwd(),name,version)
    
    def close_extension_tab(self):
        try:
            self.get_Xpath_value(xpath_Id='//p[@class="installed-loading__lead-paragraph"]')
            self.driver.switch_to.window(self.driver.window_handles[-1])  
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[-1])
        except:
            print('Disk Full ....Free Up Some Space')
            sys.exit(0)
    
    # Multi Processing Execute Function
            
    def stop_exc(self,mesg=None):
        print(mesg)
        input()
        self.close()
        sys.exit(0)
    
    def execute(self):        
        try:
            links = pd.read_excel(f"{self.file}_link.xlsx")['Links']
            self.file = f"{self.file}_link.xlsx"
            print("\nExtraction Resumes...")
        except:
            links = pd.DataFrame()
            self.scrape_links()

        self.driver.close()        
        
        if links.size == 0:
            links = pd.read_excel(f"{self.file}.xlsx")['Links']

        if links.size > 10 and links.size < 20:
            num_processes = 2
        elif links.size <= 5:
            num_processes = 1
        else:
            num_processes = int(multiprocessing.cpu_count()) // 2
            num_processes += 2
        
        num_processes = 1
        
        if links.size == 0:
            print("No Links Found To Extract Data.....")
            return        

        url_chunks = self.chunk_list(links,num_processes)        
        instance = self.__class__()

        print("Relax Now ...Extracting Data From the Links.\n")

        with multiprocessing.Pool(processes=num_processes) as pool:            
            pool.starmap(instance.scrape_data, [(instance, url_chunk) for url_chunk in url_chunks])

        print("Extraction Completed\n")
        self.remove_file()
        self.clean_temp()
        # self.collateTemfiles()

    def clean_temp(self):
        temp_path = os.environ.get('TMP')
        temp_files  = os.listdir(temp_path)
        if temp_files:
            for file in temp_files:
                try:                  
                    if os.path.isdir(os.path.join(temp_path,file)):
                        shutil.rmtree(os.path.join(temp_path,file))
                    else:
                        os.remove(os.path.join(temp_path,file))
                except Exception as ex:
                    continue                   

    def collateTemfiles(self):
        list_of_xl_dataframe = list()
        path = os.path.join(os.getcwd(),'Data')
        list_dir = os.listdir(path)

        for file in list_dir:
            list_of_xl_dataframe.append(pd.read_excel(f"{path}/{file}"))
        
        col_xl = pd.concat(list_of_xl_dataframe,ignore_index=True)
        col_xl.to_excel(f"Data/{self.file}",index=False)

        shutil.rmtree(path=path)






In [ ]:
from CustomSelenuim import customselenium
import multiprocessing,threading




class Addidas(customselenium):
    
    def __init__(self) -> None:
        self.links = list()
        self.file_lock = multiprocessing.Manager().Lock()
        self.base_link = "https://www.adidas.com"
        super().__init__()

    
    def closepopup(self,driver):
        while True:
            driver.click(xpath="//button[@name='account-portal-close']",wait_time=1)
    
    def scrape_links(self):       
        
        soup = self.get_page_source()
        links = soup.select('a.glass-product-card__assets-link')        
        self.links += [f"{self.base_link}{link.attrs.get('href')}" for link in links if link and "href" in link.attrs]

            
        link = {
            'Links' : self.links,
        }

        self.save(name=f"{self.file}_link", trig=True,**link)
        print("Links Extractions Completed \n")

    def getQualityScore(self,quality):
        if quality and "style" in quality.attrs:
               value = quality.attrs.get('style').split(';')[1].split(':')[-1]
               if "%" in value:
                   value = value.replace('%','')
               return round(float(value))
        
    def scrape_data(self,instance,url_chunk):
        instance.intialize_driver()
        new_thread = threading.Thread(target=instance.closepopup,args=(instance,),name="popup_close")
        new_thread.start()

        for link in url_chunk:
            if not link:
                continue
            
            data = {}
            
            instance.driver.get(url=link.replace('us/en/',''))            
            instance.click(xpath="//div[@class='reviews-header___1kFSZ']",wait_time=30)
            instance.click(xpath="//div[@id='navigation-target-specifications']/div",wait_time=30)
            soup = instance.get_page_source()
            category = soup.select_one('div[data-auto-id="product-category"]')
            title = soup.select_one('h1.name___120FN > span')
            price = soup.select_one('div.product-price___2Mip5 > div > div > div > div')
            rating = soup.select_one('div.ratings-label-container___13pr- > span')
            code = soup.select_one('ul.gl-list:nth-of-type(2) li:last-of-type')
            color = soup.select_one('ul.gl-list:nth-of-type(2) li:nth-last-of-type(2)')
            sizes = soup.select('button[class="gl-label size___2lbev"] span')
            review = soup.select_one('h2.accordion-title___2sTgR')
            cmf_quality = soup.select_one('div.sub-ratings___1pAhV > div:nth-of-type(1) > div > div:nth-of-type(2) > div:nth-of-type(1)')
            quality = soup.select_one('div.sub-ratings___1pAhV > div:nth-of-type(2) > div > div:nth-of-type(2) > div:nth-of-type(1)')
            size_quality = soup.select_one('div.sub-ratings___1pAhV > div:nth-of-type(3) > div > div:nth-of-type(2) > div:nth-of-type(1)')
            width_quality = soup.select_one('div.sub-ratings___1pAhV > div:nth-of-type(4) > div > div:nth-of-type(2) > div:nth-of-type(1)')

            cmf_quality = instance.getQualityScore(cmf_quality)
            if  cmf_quality is not None:

                if cmf_quality  > 50:
                    cmf_quality = f"Comfortable {cmf_quality}%"
                else:
                    cmf_quality = f"Uncomfortable {cmf_quality}%"
            else:
                cmf_quality = "--"
                

            quality = instance.getQualityScore(quality)
            if quality:
                if quality > 50:
                    quality = f"Perfect {quality}% "
                else:
                    quality = f"Poor {quality}%"
            else:

                cmf_quality = "--"

            size_quality = instance.getQualityScore(size_quality)
            if size_quality:
                if size_quality < 34:
                    size_quality = f"Too small {size_quality}%"
                elif size_quality > 34 and size_quality < 68:
                    size_quality = f"Perfect {size_quality}%"
                else:
                    size_quality = f"Too large {size_quality}%"
            else:
                size_quality = '--'

            
            width_quality = instance.getQualityScore(width_quality)
            if width_quality:
                if width_quality < 34:
                    width_quality = f"Too narrow {width_quality}%"
                elif width_quality > 34 and width_quality < 68:
                    width_quality = f"Perfect {width_quality}%"
                else:
                    width_quality = f"Too wide {width_quality}%"
            else:
                width_quality = '--'

            data = {
                'Category' : category.text if category else None,
                'Title': title.text if title else None,
                'Price': price.text if price else None,
                'Code': code.text if code else None,                
                'Available Size': ' '.join([size.text for size in sizes if size]),
                'Rating' : rating.text if rating else None,                
                'No of Reviews' : review.text if review else None,
                'No of Size':len(sizes) if sizes else 0,
                'No of Color': len(color.next.split('/')) if color else 0, 
                'Comfort' : cmf_quality,
                'Quality' : quality,
                'Size' : size_quality,
                'Width' : width_quality
            }

            with instance.file_lock:
                self.save(**data)                

        instance.close()

if __name__ == "__main__":
    url = "https://www.adidas.com/us/women-shoes"
    add = Addidas()
    add.intialize_driver()
    add.driver.get(url=url)
    add.execute()


In [ ]:
from CustomSelenuim import customselenium
import multiprocessing


class Puma(customselenium):
    def __init__(self) -> None:
        self.links = list()
        self.base_link = "https://us.puma.com/"
        self.file_lock = multiprocessing.Manager().Lock()
        super().__init__()
    
    def scrape_links(self):        

            
        soup = self.get_page_source()
        links = soup.select('li[data-test-id="product-list-item"] a')
        self.links += [f"{self.base_link}{link.attrs.get('href')}" for link in links if link and "href" in link.attrs]
        
        link = {
            'Links' : self.links,
        }
        self.save(name=f"{self.file}_link", trig=True,**link)
        print("Links Extractions Completed \n")
    
    def scrape_data(self,instance,url_chunk):
        
        instance.intialize_driver()
        for link in url_chunk:
            
            if not link:
                continue
            
            data = {}
            
            instance.driver.get(url=link.replace('us/en/',''))
            instance.click(xpath='//button[@data-test-id="read-all-reviews"]')
            self.randomSleep(2,2)
            
            soup = instance.get_page_source()
            category = soup.select_one('ul[data-uds-child="breadcrumb-list"] > li:nth-of-type(2)')
            title = soup.select_one('h1#pdp-product-title')
            price = soup.select_one('div[data-test-id="pdp-price"] span:nth-of-type(1)')
            code = soup.select_one('ul.tw-1h4nwdw.tw-p9uz4a.tw-xwzea6.list-disc.list-inside li:nth-of-type(1)')
            color = soup.select('div#style-picker label span')
            sizes = soup.select('div#size-picker label span span:last-of-type')
            review = soup.select_one('div#product-reviews > div:nth-of-type(1) > h2')
            rating = soup.select_one('p[data-test-id="percentRecommendThisProduct"]')
            
            if rating:
                rating = rating.text.split('%')[0]                
                rating = float(rating) / 10
                rating = rating / 2
            else:
                rating = 0.0
                

            data = {
                'Category': category.text if category else None,
                'Title': title.text if title else None,
                'Price': price.text if price else None,
                'Code': code.text if code else None,
                'Available Size': ' '.join([size.text for size in sizes if size]),     
                'No of Size':len(sizes) if sizes else 1,
                'No of Color': len(color) if color else 0,
                'review' : review.text if review else None,
                'rating': rating
            
            }
            
            with instance.file_lock:
                self.save(**data)

        instance.close()

    


if __name__ == "__main__":
    puma = Puma()
    puma.intialize_driver()
    puma.driver.get(url="https://us.puma.com/us/en/women/shoes") # https://us.puma.com/us/en/men/shop-all-mens#
    puma.execute()

        


In [ ]:
from CustomSelenuim import customselenium 
import multiprocessing


class Nike(customselenium):
    def __init__(self) -> None:
        self.links = list()
        self.file_lock = multiprocessing.Manager().Lock()
        super().__init__()

    def scrape_links(self):        

        soup = self.get_page_source()
        
        links = soup.select('a.product-card__link-overlay')
        self.links += [link.attrs.get('href') for link in links if link and "href" in link.attrs]
        
        link = {
            'Links' : self.links,
        }

        self.save(name=f"{self.file}_link", trig=True,**link)
        print("Links Extractions Completed \n")

    def scrape_data(self,instance,url_chunk):
        
        instance.intialize_driver()
        for link in url_chunk:
            if not link:
                continue
            
            data = {}
            
            instance.driver.get(url=link)

            instance.click(xpath='//details[@data-test="reviewsAccordionClick"]')

            soup = instance.get_page_source()
            
            title = soup.select_one('h1[data-test="product-title"]')
            price = soup.select_one('div[data-test="product-price"]')
            category = soup.select_one('h2[data-test="product-sub-title"]')
            code = soup.select_one('li.description-preview__style-color')
            color = soup.select('div.colorway-images-wrapper fieldset div div label img')
            
            if not color:
                color = soup.select_one('li.description-preview__color-description.ncss-li')
                color = len(color.next.split('/')) if color else 0
            else:
                color = len(color)


            sizes = soup.select('label.css-xf3ahq')
            review = soup.select_one('details[data-test="reviewsAccordionClick"] summary h3')
            rating = soup.select_one('details[data-test="reviewsAccordionClick"] summary div')

            data = {
                'Category': category.text if category else None,
                'Title': title.text if title else None,
                'Price': price.text if price else None,
                'Code': code.text if code else None,             
                'Available Size': ' '.join([size.text for size in sizes if size]),
                'No of Size':len(sizes) if sizes else 1,
                'No of Color': color, 
                'review' : review.text if review else None,
                'rating':rating.attrs.get('aria-label') if rating else 0.0
            }

            with instance.file_lock:
                self.save(**data)

        instance.close()


if __name__ == "__main__":

    #Just Pass the url of category it will extract the data
    # default url is passed men's shoes
    
    url = "https://www.nike.com/in/w/womens-shoes-5e1x6zy7ok" 
    nik = Nike()
    nik.intialize_driver()
    nik.driver.get(url=url)
    nik.execute()

In [ ]:
from CustomSelenuim import customselenium 
import multiprocessing

class Skeechers(customselenium):
    def __init__(self) -> None:
        self.links = list()
        self.base_url = "https://www.skechers.com"
        self.file_lock = multiprocessing.Manager().Lock()
        super().__init__()

    def scrape_links(self):        
               
        soup = self.get_page_source()
        links = soup.select('div.product > div > div > a')  
        self.links += [f"{self.base_url}{link.attrs.get('href')}"for link in links if link and "href" in link.attrs]
                
        link = {
            'Links' : self.links,
        }

        self.save(name=f"{self.file}_link", trig=True,**link)
        print("Links Extractions Completed \n")

    def scrape_data(self,instance,url_chunk):
        instance.intialize_driver()
        
        for link in url_chunk:

            instance.randomSleep(2,3)
            
            if not link:
                continue
            
            data = {}
            
            instance.driver.get(url=link)
            soup = instance.get_page_source()
            category = soup.select_one('div.c-product-details__label')            
            title = soup.select_one('h1.c-product-details__product-name.product-name')
            price = soup.select_one('div.price > span > span > span > span')
            colors = soup.select('button[aria-describedby="color"]')
            sizes = soup.select('span.size-value.c-product-attributes__item__value--size.c-product-attributes__item__value.swatch-value.selectable')
            rating = soup.select_one('div.ratings > span')
            review  = soup.select_one('div.pr-snippet-read-and-write a')
            code = soup.select_one('span.product-id')

            data = {
                'Category': category.text if category else None,
                'Title': title.text if title else None,
                'Price': price.text if price else None,         
                'Code': code.text if code else None,
                'Available Size': ' '.join([size.text for size in sizes if size]),     
                'No of Size':len(sizes) if sizes else 1,                  
                'No of Color': len(colors) if colors else 1,
                'review' : review.text if review else "0 Reviews",
                'Rating' :rating.text if rating else None,
            }

            with instance.file_lock:
                self.save(**data)

        instance.close()


if __name__ == "__main__":

    #Just Pass the url of category it will extract the data
    # default url is passed men's shoes
    
    url = "https://www.skechers.com/women/shoes/" 
    sk = Skeechers()
    sk.intialize_driver()
    sk.driver.get(url=url)
    sk.execute()